In [1]:
from bs4 import BeautifulSoup
import requests
from lxml import etree
import csv
import time

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [6]:
def save_NoB(NoB, id_azienda, writer):
    NoB = NoB.replace(' ', '')
    NoB = NoB.replace('\n', '')
    row = [id_azienda, NoB, hash(NoB)]    
    writer.writerow(row)

In [7]:
def get_info(url, id_, writer_a, writer_n):
    pre_req = time.time()
    req = requests.get(url, headers)
    post_req = time.time()
    tot_req = post_req - pre_req
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    
    #Variabili
    global name_txt
    global c_id_txt
    global c_type_txt
    global address_txt
    global status_txt
    global registered_txt
    global incorporated_txt
    global dissolved_txt
    
    #Company name
    pre_name = time.time()
    name = dom.xpath('//p[@class = "heading-xlarge"]')
    post_name = time.time()
    tot_name = post_name - pre_name
    if name:
        name_txt = name[0].text
    else:
        name_txt = None
    
    #Company number
    pre_id = time.time()
    c_id = dom.xpath('//p[@id = "company-number"]/strong')
    post_id = time.time()
    tot_id = post_id - pre_id
    if c_id:
        c_id_txt = c_id[0].text
    else:
        c_id_txt = None
    
    #Company type
    pre_type = time.time()
    c_type = dom.xpath('//dt[text() = "Company type"]/../dd')
    post_type = time.time()
    tot_type = post_type - pre_type
    if c_type:
        c_type_txt = c_type[0].text
        c_type_txt = c_type_txt.replace(' ', '')
        c_type_txt = c_type_txt.replace('\n', '')
    else:
        c_type_txt = None
    
    #Registered office address
    if(c_type_txt == 'Overseasentity'):
        pre_add = time.time()
        address = dom.xpath('//dt[text() = "Correspondence address"]/../dd')
        post_add = time.time()
        tot_add = post_add - pre_add
        if address:
            address_txt = address[0].text
            address_txt = address_txt.replace('\n', '')
            address_txt = address_txt.replace(' ', '')
            address_txt = address_txt.replace(',', ' ')
        else:
            address_txt = None
    else:
        pre_add = time.time()
        address = dom.xpath('//dt[text() = "Registered office address"]/../dd')
        post_add = time.time()
        tot_add = post_add - pre_add
        if address:
            address_txt = address[0].text
            address_txt = address_txt.replace('\n', '')
            address_txt = address_txt.replace(' ', '')
            address_txt = address_txt.replace(',', ' ')
        else:
            address_txt = None
        
    #Company status
    pre_status = time.time()
    status = dom.xpath('//dt[text() = "Company status"]/../dd')
    post_status = time.time()
    tot_status = post_status - pre_status
    if status:
        status_txt = status[0].text 
        status_txt = status_txt.replace(' ', '')
        status_txt = status_txt.replace('\n', '')
    else:
        status_txt = None
    
    #Registration Date
    if(status_txt == 'Registered'):
        pre_date = time.time()
        registered = dom.xpath('//dt[text() = "Registered on"]/../dd')
        post_date = time.time()
        tot_date = post_date - pre_date
        incorporated_txt = None
        if registered:
            registered_txt = registered[0].text
        else:
            registered_txt = None
    else:
        pre_date = time.time()
        incorporated = dom.xpath('//dt[text() = "Incorporated on"]/../dd')
        post_date = time.time()
        tot_date = post_date - pre_date
        registered_txt = None
        if incorporated:
            incorporated_txt = incorporated[0].text
    
    #Dissolved on
    if(status_txt == 'Dissolved'):
        pre_dateD = time.time()
        dissolved = dom.xpath('//dt[text() = "Dissolved on"]/../dd')
        post_dateD = time.time()
        tot_dateD = post_dateD - pre_dateD
        if dissolved:
            dissolved_txt = dissolved[0].text
        else:
            dissolved_txt = None
    else:
        dissolved_txt = None
        tot_dateD = 0
    
    #Nature of business
    pre_NoB = time.time()
    NoB = dom.xpath('//span[@id = "sic0"]/text()')
    post_NoB = time.time()
    tot_NoB = post_NoB - pre_NoB
    for elem in NoB:
        save_NoB(elem, id_, writer_n)
    
    #Save Line
    line = [url, id_, name_txt, c_id_txt, status_txt, c_type_txt, registered_txt, 
            incorporated_txt, dissolved_txt, address_txt, tot_req, tot_name, tot_id, tot_status,
            tot_type, tot_date, tot_dateD, tot_add, tot_NoB]
    writer_a.writerow(line)

In [20]:
url = 'https://find-and-update.company-information.service.gov.uk'
ds_a = open('govuk-aziende-dataset.csv', 'w')
head_a = ['URL', 'ID','Name', 'Company ID', 'Company Status', 'Company Type', 
        'Registration Date', 'Incorporation Date', 'Dissolution Date', 
        'Office Address', 'Tempo richiesta', 'Tempo estrazione Name', 'Tempo estrazione Company ID',
        'Tempo estrazione Company Status','Tempo estrazione Company Type',
        'Tempo estrazione Reg/Inc Date', 'Tempo estrazione Diss Date', 'Tempo estrazione Address',
        'Tempo estrazione Nature of Business']
writer_a = csv.writer(ds_a)
writer_a.writerow(head_a)

ds_n = open('govuk-NoB-dataset.csv', 'w')
head_n = ['ID Azienda', 'Nature of Business']
writer_n = csv.writer(ds_n)
writer_n.writerow(head_n)

d = {}
id_ = 0

#RICERCA SU 'a'
start = time.time()
for x in range(1, 200):
    url2 = url + '/search/companies?q=p&page=' + str(x)
    start_req = time.time()
    req = requests.get(url2, headers)
    end_req = time.time()
    tot_req = end_req - start_req
    soup = BeautifulSoup(req.content, 'html.parser')
    d[x] = [tot_req]
    print(x)
    for a in soup.find_all('a', class_ = 'govuk-link', href = True):
        if '/company' in a['href']:
            id_ += 1
            get_info(url + a['href'], id_, writer_a, writer_n)

ds_a.close()
ds_n.close()
end = time.time();
print(end - start)
print(id_, 'aziende')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
34.82701849937439
0 aziende


In [15]:
print(d)

{1: [0.567274808883667], 2: [0.6212272644042969], 3: [0.5601036548614502], 4: [0.6229991912841797], 5: [0.5825843811035156], 6: [0.5724349021911621], 7: [0.1801621913909912], 8: [0.16029834747314453], 9: [0.1646120548248291], 10: [0.17130064964294434], 11: [0.17657876014709473], 12: [0.1701204776763916], 13: [0.2035233974456787], 14: [0.1735842227935791], 15: [0.16440606117248535], 16: [0.16233158111572266], 17: [0.16658329963684082], 18: [0.16609930992126465], 19: [0.17107057571411133], 20: [0.1673743724822998], 21: [0.16472911834716797], 22: [0.17299151420593262], 23: [0.17553162574768066], 24: [0.1809673309326172], 25: [0.17486000061035156], 26: [0.15187597274780273], 27: [0.1574094295501709], 28: [0.17794108390808105], 29: [0.17278528213500977], 30: [0.17521286010742188], 31: [0.16750478744506836], 32: [0.18411898612976074], 33: [0.1766352653503418], 34: [0.17024016380310059], 35: [0.17641305923461914], 36: [0.1684403419494629], 37: [0.16993260383605957], 38: [0.1647014617919922], 